In [22]:
# Importing the required libraries and dependencies
import numpy as np
import pandas as pd
import hvplot.pandas
import sqlalchemy
from sqlalchemy import inspect

# Create a temporary SQLite database and populate the database with content from the bank seed file
database_connection_string = 'sqlite:///bank'

# Create an engine to interact with the SQLite database
engine = sqlalchemy.create_engine(database_connection_string)

# Use the inspect module to confirm the table names contained in the SQLite database.
inspector = inspect(engine)
table_names = inspector.get_table_names()

print("Tables in the database:", table_names)


Tables in the database: ['BIDV', 'MB', 'TCB', 'VCB']


## Analyze a single asset in the Bank


In [23]:
# Write a SQL query to SELECT all of the data from the TCB table
query = """
SELECT *
FROM TCB
"""

# Use the query to read the TCB data into a Pandas DataFrame
tcb_dataframe = pd.read_sql_query(query, con=engine)

In [24]:
# View the first 5 rows of the DataFrame.
tcb_dataframe.head()


,time,open,high,low,close,volume,daily_return
0,2024-05-21,48100.00,48200.00,47050.00,47200.00,11258500.0,-0.02
1,2024-05-20,48442.45,48975.86,48054.53,48200.00,11449601.0,0.00
2,2024-05-17,48248.49,48490.95,48151.51,48248.49,6610750.0,0.00
3,2024-05-16,47036.22,48442.45,46939.24,48393.96,21363102.0,0.04
4,2024-05-15,46454.33,46696.78,46308.85,46648.29,10026100.0,0.00


In [25]:
# View the last 5 rows of the DataFrame.
tcb_dataframe.tail()


,time,open,high,low,close,volume,daily_return
996,2020-05-26,20463.18,20657.14,20366.20,20511.67,2830450.0,0.00
997,2020-05-25,20463.18,20657.14,20317.71,20414.69,2540930.0,0.00
998,2020-05-22,20948.09,20948.09,20414.69,20414.69,3672780.0,-0.03
999,2020-05-21,20414.69,21336.02,20220.72,21336.02,3539410.0,0.05
1000,None,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# Create an interactive visualization with hvplot to plot the daily returns for PYPL.
tcb_dataframe['daily_return'].hvplot(
    title = "Daily Return",
    x = "time",
    xlabel = "Time (specified in days)",
    ylabel = "Daily Returns",
)


:Curve   [time]   (daily_return)

In [27]:
# Create an interactive visaulization with hvplot to plot the cumulative returns for PYPL.
tcb_dataframe['daily_return'].cumsum().hvplot(
    title = "Daily Return",
    x = "time",
    xlabel = "Time (specified in days)",
    ylabel = "Daily Returns",
)


:Curve   [time]   (daily_return)

## Optimize the SQL Queries


In [28]:
# Write a SQL SELECT statement to select the time and close columns 
# where the PYPL closing price was higher than 30000
query = """
SELECT time, close
FROM TCB
WHERE close >= 30000
"""

# Using the query, read the data from the database into a Pandas DataFrame
tcb_higher_than_30000 = pd.read_sql(query, con=engine)

# Review the resulting DataFrame
display(tcb_higher_than_30000.head())

,time,close
0,2024-05-21,47200.00
1,2024-05-20,48200.00
2,2024-05-17,48248.49
3,2024-05-16,48393.96
4,2024-05-15,46648.29


In [29]:
# Sort the results in descending order and return only the top 10 return values
query = """
SELECT time, close
FROM TCB
ORDER BY daily_return DESC
LIMIT 10;
"""

# Using the query, read the data from the database into a Pandas DataFrame
tcb_top_10_returns = pd.read_sql(query, con=engine)

# Review the resulting DataFrame
display(tcb_top_10_returns.head())


,time,close
0,2022-11-16,21433.00
1,2021-05-04,42090.14
2,2021-01-29,31034.21
3,2024-03-21,43738.83
4,2022-12-02,27930.78


## Analyze the Fintech Bank



In [30]:
# Use the time column from each table as the basis for the join
query = """
SELECT *
FROM TCB
JOIN MB ON TCB.time = MB.time
JOIN BIDV ON TCB.time = BIDV.time
JOIN VCB ON TCB.time = VCB.time
"""

# Using the query, read the data from the database into a Pandas DataFrame
bank_portfolio = pd.read_sql(query, con=engine)

# Review the resulting DataFrame
display(bank_portfolio.head())


,time,open,high,low,close,volume,daily_return,time,open,high,...,close,volume,daily_return,time,open,high,low,close,volume,daily_return
0,2024-05-21,48100.00,48200.00,47050.00,47200.00,11258500.0,-0.02,2024-05-21,23450.0,23600.0,...,50000.0,1794700.0,0.01,2024-05-21,91900.0,92000.0,90700.0,91100.0,1486400.0,-0.01
1,2024-05-20,48442.45,48975.86,48054.53,48200.00,11449601.0,0.00,2024-05-20,23600.0,23600.0,...,49600.0,2191900.0,0.01,2024-05-20,92300.0,93000.0,91800.0,91900.0,1043800.0,0.00
2,2024-05-17,48248.49,48490.95,48151.51,48248.49,6610750.0,0.00,2024-05-17,23450.0,23450.0,...,49350.0,1518300.0,-0.01,2024-05-17,92600.0,92600.0,91700.0,91800.0,1084700.0,-0.01
3,2024-05-16,47036.22,48442.45,46939.24,48393.96,21363102.0,0.04,2024-05-16,23150.0,23550.0,...,49600.0,2097300.0,0.02,2024-05-16,92000.0,92700.0,91500.0,92600.0,1713900.0,0.02
4,2024-05-15,46454.33,46696.78,46308.85,46648.29,10026100.0,0.00,2024-05-15,22950.0,23050.0,...,48600.0,1393000.0,0.01,2024-05-15,91000.0,91300.0,90800.0,91000.0,883100.0,0.00


In [31]:
# Create a DataFrame that displays the mean value of the “daily_returns” columns for all four assets.
bank_portfolio_returns = bank_portfolio['daily_return'].mean(axis=1)

# Review the resulting DataFrame
bank_portfolio_returns


0     -0.0025
1      0.0050
2     -0.0075
3      0.0250
4      0.0025
        ...  
994    0.0100
995   -0.0275
996    0.0225
997    0.0025
998   -0.0125
Length: 999, dtype: float64

In [32]:
# to calculate the annualized return for the portfolio. 
annualized_bank_portfolio_returns = bank_portfolio_returns.mean() * 252

# Display the annualized return value of the ETF portfolio.
annualized_bank_portfolio_returns


0.07819819819819819

In [33]:
# to calculate the cumulative returns
bank_cumulative_returns = bank_portfolio_returns.cumsum()

# Display the final cumulative return value
display(bank_cumulative_returns)


0     -0.0025
1      0.0025
2     -0.0050
3      0.0200
4      0.0225
        ...  
994    0.3250
995    0.2975
996    0.3200
997    0.3225
998    0.3100
Length: 999, dtype: float64

In [34]:
# Using hvplot, create an interactive line plot that visualizes the bank portfolios cumulative return values.
bank_cumulative_returns.hvplot(
    title = "Cumulative Returns",
    x = "time",
    xlabel = "Time (specified in days)",
    ylabel = "Cumulative Returns",
    yformatter = '%.1f'
)


:Curve   [time]   (0)